<a href="https://colab.research.google.com/github/OlesiaPoliakova/Education/blob/main/PyTestParametrization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![alt text](https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg)      PYTHON BASICS

##![parametrization](https://zappysys.com/onlinehelp/ssis-powerpack/scr/images/csv-source/ssis-csv-file-source.png) PARAMETRIZED TESTS

In [ ]:
#@title Installeng dependencies

!pip install pytest ipython_pytest ipyext unittest2
%load_ext ipython_pytest
%load_ext ipyext.writeandexecute
!rm -rf work_imports
!mkdir work_imports
!touch work_imports/__init__.py

###![bank acount](https://cdn0.iconfinder.com/data/icons/banking-color/64/open-account-bank-passbook-128.png) Bank Account Management App

In [ ]:
%%writeandexecute -i identifier work_imports/card_kinds.py
from enum import Enum


class CardsKinds(Enum):
    NO_CARD = "No Card"
    NORMAL_CARD = "Normal Card"
    GOLD_CARD = "Gold Card"
    PLATINUM_CARD = "Platinum Card"

In [ ]:
%%writeandexecute -i identifier work_imports/exceptions.py

class IncorrectDebetError(Exception):
    pass

class TakeMoneyUnavailabilityError(Exception):
    pass

In [ ]:
%%writeandexecute -i identifier work_imports/bank_support.py
from random import randint
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

class BankSupport(object):
    def __init__(self, account_debet=0.0):
        self.account_id = randint(10000, 99999)
        self.account_debet = account_debet

    @property
    def card_kind(self):
        if self.account_debet < 0.0:
            raise IncorrectDebetError
            (
                f"Bank account #{str(self.account_id)}. \
                Debet is under zero: {str(self.account_debet)}"
            )
        elif self.account_debet == 0.0:
            return CardsKinds.NO_CARD
        elif self.account_debet > 0.0 and \
        self.account_debet <= 9000.0:
            return CardsKinds.NORMAL_CARD
        elif self.account_debet > 10000.0 and \
        self.account_debet <= 99999.0:
            return CardsKinds.GOLD_CARD
        return CardsKinds.PLATINUM_CARD

    def charge_account(self, sum: float):
        if self.account_debet < 0.0:
            raise IncorrectDebetError
            (
                f"Bank account #{str(self.account_id)}. \
                Debet is under zero: {str(self.account_debet)}"
            )

    def take_founds(self, sum: float):
        if self.account_debet < 0.0:
            raise IncorrectDebetError
            (
                f"Bank account #{str(self.account_id)}. \
                Debet is under zero: {str(self.account_debet)}"
            )
        elif self.account_debet < sum:
            raise TakeMoneyUnavailabilityError
            (
                f"Bank account #{str(self.account_id)}. \
                Sum to take {str(sum)} is more than your debet: \
                {str(self.account_debet)}"
            )
        self.account_debet -= sum

### Simple Parametrized Tests

In [ ]:
%%pytest
import pytest
from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

#pytest -v --setup-show --debug 
#test/pytest/TestBankSupportWithFixtureScopeTuning.py

class Config(object):
    card_provider = None
    account_provider = None

@pytest.fixture(scope="function")
def check_exception():
    def checker(function, exception: Exception, *args):
        try:
            print
            (
                f"Checking for exception {exception} \
                for function {function.__name__}"
            )
            
            function(*args)
            
        except exception:
            return True
        
        except Exception:
            return False
        
        return False
    
    return checker

@pytest.fixture(scope="session", autouse=True)
def account_provider():
    print(f"Providing Banking Account {BankSupport}")
    Config.account_provider = BankSupport

@pytest.fixture(scope="module", autouse=True)
def card_provider():
    print("Providing Card types enumeration")
    Config.card_provider = CardsKinds


@pytest.mark.parametrize("funds, expected_card_kind", 
                         ((0, CardsKinds.NO_CARD),
                         (1, CardsKinds.NORMAL_CARD),
                         (10001, CardsKinds.GOLD_CARD),
                         (100010, CardsKinds.PLATINUM_CARD)))
def test_card(funds: int, expected_card_kind: CardsKinds):
    card_kind = Config.account_provider(funds).card_kind
    assert card_kind == expected_card_kind, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {expected_card_kind}"

@pytest.mark.parametrize("method, param, exception", 
                         (
                             (BankSupport, -10, 
                              IncorrectDebetError),
                             (BankSupport(100).take_founds, 
                              101, 
                              TakeMoneyUnavailabilityError)
                         )
                        )
def test_negative(
    check_exception, 
    method, 
    param, 
    exception: Exception):
    assert check_exception(method, exception, param), \
f"Exception {exception.__name__} hasn't been raised"

### Tests with generated parameters (Data Provider)

In [ ]:
%%pytest
import pytest
import random
from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

#pytest -v --setup-show 
#--debug test/pytest/TestBankSupportWithFixtureScopeTuning.py

class Config(object):
    card_provider = None
    account_provider = None


class _account_card_type_data_provider(object):
    def __init__(self):
        self.cards = (
            CardsKinds.NO_CARD, 
            CardsKinds.NORMAL_CARD, 
            CardsKinds.GOLD_CARD, 
            CardsKinds.PLATINUM_CARD
        )
        
        print(f"Cards: {self.cards}")

    def __call__(
        self, 
        normal_threshold, 
        gold_threshold, 
        platinum_threshold
    ):
        tmp = list()
        thresholds = (
                      (0, 0),
                      (1, normal_threshold),
                      (normal_threshold + 1, gold_threshold),
                      (gold_threshold + 1, platinum_threshold)
                      )

        for i in self.cards:
            ind = thresholds[self.cards.index(i)]
            for _ in range(3):
                print(f"{str(ind[0])}: {str(ind[1])} -> {i}")
                tmp.append((random.randint(ind[0], ind[1]), i))
        return tuple(tmp)

account_card_type_data_provider = _account_card_type_data_provider()
    

@pytest.fixture(scope="function")
def check_exception():
    def checker(function, exception: Exception, *args):
        try:
            print
            (
                f"Checking for exception {exception} for \
                function {function.__name__}"
            )
            function(*args)
        except exception:
            return True
        except Exception:
            return False
        return False
    return checker

@pytest.fixture(scope="session", autouse=True)
def account_provider():
    Config.account_provider = BankSupport

@pytest.fixture(scope="module", autouse=True)
def card_provider():
    Config.card_provider = CardsKinds


@pytest.mark.parametrize("funds, expected_card_kind", 
                         account_card_type_data_provider(10000, 
                          100000, 
                          10000000)
                        )
def test_card(funds: int, expected_card_kind: CardsKinds):
    card_kind = Config.account_provider(funds).card_kind
    assert card_kind == expected_card_kind, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {expected_card_kind}. Funds: {str(funds)}"

@pytest.mark.parametrize("method, param, exception", (
    (BankSupport, -10, IncorrectDebetError),
    (
        BankSupport(100).take_founds, 
        101, 
        TakeMoneyUnavailabilityError
    )
)
                        )
def test_negative_debet(check_exception, 
                        method, 
                        param, 
                        exception: Exception):
    assert check_exception(method, exception, param), \
    f"Exception IncorrectDebetError hasn't been raised"

### Mixing PyUnit and PyTest in one suite using DataProviders

In [ ]:
%%pytest
import pytest
import random
import unittest
from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

@pytest.fixture(scope="function")
def check_exception():
    def checker(function, exception: Exception, *args):
        try:
            print
            (
                f"Checking for exception {exception} for \
                function {function.__name__}"
            )
            
            function(*args)
            
        except exception:
            return True
        
        except Exception:
            return False
        
        return False
    
    return checker

@pytest.fixture(scope="class")
def account_provider(request):
    class Config(object):
        def __init__(self):
            self.bank_account = BankSupport
            self.cards = CardsKinds

    request.cls.config = Config()

@pytest.mark.usefixtures("account_provider")
class MyTestSuite(unittest.TestCase):

    def test_no_card(self):
        card_kind = self.config.bank_account(0).card_kind
        self.assertEqual
        (
            card_kind, 
            self.config.cards.NO_CARD, 
            f"Incorrect type of card has been returned: \
            {card_kind}. \
            Should be {self.config.cards.NO_CARD}"
        )

    def test_normal_card(self):
        card_kind = self.config.bank_account(1).card_kind
        self.assertEqual
        (
            card_kind, 
            self.config.cards.NORMAL_CARD,
            f"Incorrect type of card has been returned: \
            {card_kind}. \
            Should be {self.config.cards.NORMAL_CARD}"
        )
        
    def test_gold_card(self):
        card_kind = self.config.bank_account(10001).card_kind
        self.assertEqual
        (
            card_kind, 
            self.config.cards.GOLD_CARD,
            f"Incorrect type of card has been returned: \
            {card_kind}. \
            Should be {self.config.cards.GOLD_CARD}"
        )
        
    def test_platinum(self):
        card_kind = self.config.bank_account(100001).card_kind
        self.assertEqual
        (
            card_kind, 
            self.config.cards.PLATINUM_CARD,
            f"Incorrect type of card has been returned: \
            {card_kind}. \
            Should be {self.config.cards.PLATINUM_CARD}"
        )
        
    def test_negative_debet(self):
        negative_debet = -10.0
        with self.assertRaises(IncorrectDebetError):
            self.config.bank_account(negative_debet)

    def test_take_more_then(self):
        normal_account = self.config.bank_account(100)
        with self.assertRaises(TakeMoneyUnavailabilityError):
            normal_account.take_founds(1001)

### Data Provider which reads test data from the external file

In [ ]:
#@title Mount of the Google Drive
from google.colab import drive
drive.mount('/content/drive')
!pip install pandas

In [ ]:
%%pytest
import pandas

from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

def pytest_generate_tests(metafunc):
  test_data = tuple(pandas.read_csv("/content/drive/My Drive/Colab Notebooks/a4q_selenium/PythonPart/PyTest/test_data.csv").itertuples(index=False, name=None))
  
  metafunc.parametrize("funds, expected_card_kind", test_data)

class TestBankAccount(object):
    def test_card(self, funds: int, 
                  expected_card_kind: CardsKinds): 
        card_kind = BankSupport(funds).card_kind
        assert card_kind == CardsKinds(expected_card_kind), \
        f"Incorrect type of card has been returned: {card_kind}. \
        Should be {CardsKinds(expected_card_kind)}. Funds: {str(funds)}"